In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T
from pyspark.conf import SparkConf
import pandas as pd

#path = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ab.gz'
start_date = 1800
end_date = 2000
directory_3gram = 'C:/Users/bincl/BA-Thesis/Dataset/3gram/parquet/3_20000_nopos_sample/3_20000_nopos_sample.gz/'
directory_2gram = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/parquet/orderBy/2gram_order/'

conf= SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','16g'),('spark.sql.sources.pushdown.pushdownEnabled', 'true')])
spark = SparkSession.builder.config(conf=conf).appName('NgramSQL').getOrCreate()

#spark = SparkSession.builder.appName('3gramSQL').getOrCreate()
df_3gram = spark.read.parquet(directory_3gram)   
df_2gram = spark.read.parquet(directory_2gram)

In [2]:
def get_pd_df(df):
    matched = df['Data'] 
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(start_date,end_date))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

def get_pd_df_from_string(suchstring, df):
    x = df.select(['Tokens', 'Data']).where(df.Tokens == suchstring).limit(1)
    matched = x.first()['Data']
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(start_date,end_date))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

In [3]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   if len(middle) != 0:
        first = first + " " + ''.join(middle)
        last = ''.join(middle) + " " + last    
   return first, last

In [4]:
df = df_3gram.head(10)
print(df[1])

Row(Tokens='Erkenntnisse und Methoden', Data={'1855': 1, '1976': 217, '1975': 167, '1974': 211, '1973': 239, '1851': 3, '1972': 174, '1971': 212, '1970': 205, '1979': 237, '1978': 212, '1977': 219, '1909': 11, '1908': 1, '1990': 245, '1987': 202, '1986': 203, '1985': 235, '1984': 237, '1983': 316, '1982': 263, '1981': 279, '1980': 312, '1906': 18, '1905': 3, '1903': 3, '1902': 3, '1901': 4, '1989': 201, '1900': 1, '1988': 215, '1910': 12, '1998': 205, '1876': 3, '1997': 169, '1996': 153, '1874': 2, '1995': 148, '1994': 205, '1993': 183, '1992': 192, '1991': 205, '1918': 3, '1917': 11, '1916': 4, '1914': 9, '1913': 22, '1912': 1, '1911': 4, '1999': 177, '1888': 8, '1921': 8, '1920': 8, '1929': 18, '1928': 10, '1927': 28, '1926': 5, '1925': 4, '1924': 5, '1923': 2, '1922': 4, '1932': 9, '1898': 2, '1931': 7, '1897': 4, '1930': 18, '1896': 1, '1939': 12, '1938': 15, '1937': 17, '1936': 12, '1935': 17, '1934': 24, '1933': 24, '2001': 153, '2000': 169, '1943': 14, '1942': 12, '1941': 12, '1

In [5]:

for d in df:
    first, last = getLowerLevelString(d["Tokens"])
    
    x = df_2gram.select(['Tokens', 'Data']).where(df_2gram.Tokens == first).limit(1)
    d_first = x.first()['Data']

    x = df_2gram.select(['Tokens', 'Data']).where(df_2gram.Tokens == last).limit(1)
    d_last = x.first()['Data'] 
    

In [6]:
df = df_3gram.head(1000)
for d in df:
    first, last = getLowerLevelString(d["Tokens"])
    
    x = df_2gram.select(['Tokens','Sum']).where(df_2gram.Tokens == first).limit(1)
    d_first = x.first()['Tokens']

    x = df_2gram.select(['Tokens','Sum']).where(df_2gram.Tokens == last).limit(1)
    d_last = x.first()['Tokens']

In [8]:
df = df_3gram.head(10)
for d in df:
    first, last = getLowerLevelString(d["Tokens"])
    
    x = df_2gram.select(['Data']).where(df_2gram.Tokens == first).limit(1)
    d_first = x.first()['Data']

    x = df_2gram.select(['Data']).where(df_2gram.Tokens == last).limit(1)
    d_last = x.first()['Data'] 

In [9]:
df_3gram = df_3gram.withColumn('last_two_words', expr("substring_index(Tokens, ' ', -2)"))
df_3gram = df_3gram.withColumn('first_two_words', expr("substring_index(Tokens, ' ', 2)"))

In [15]:
df_3gram = df_3gram.join(df_2gram.alias("last"), df_3gram['last_two_words'] == col("last.Tokens"))
joined_df = df_3gram.join(df_2gram.alias("first"), df_3gram['first_two_words'] == col("first.Tokens"))

In [16]:
joined_df.printSchema()

root
 |-- Tokens: string (nullable = true)
 |-- Data: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- Sum: integer (nullable = true)
 |-- last_two_words: string (nullable = true)
 |-- first_two_words: string (nullable = true)
 |-- Tokens: string (nullable = true)
 |-- Data: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- Sum: integer (nullable = true)
 |-- Tokens: string (nullable = true)
 |-- Data: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- Sum: integer (nullable = true)
 |-- Tokens: string (nullable = true)
 |-- Data: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- Sum: integer (nullable = true)



In [ ]:
df = joined_df.head(10)
print(df)